In [2]:
import pandas as pd

faq_test_df = pd.read_csv("faq_testset.csv")

In [3]:
faq_test_df.head()

,user_input,reference
0,FPT Shop thu thập thông tin cá nhan cua khach ...,FPTshop.com.vn thu thập thông tin cá nhân của ...
1,How does FPTshop.com.vn use customer personal ...,FPTshop.com.vn chỉ sử dụng thông tin cá nhân c...
2,Thông tin cá nhân của khách hàng được lưu trữ ...,Dữ liệu cá nhân sẽ được lưu trữ đến khi có yêu...
3,FPT Shop cam kết bảo vệ thông tin của khách hà...,FPTshop.com.vn cam kết bảo mật thông tin cá nh...
4,"As a Data Privacy Officer, I am particularly i...",FPTShop ensures that customer payment card inf...


In [32]:
from service.store_chatbot import gen_answer
from uuid import uuid4, UUID
from models.message import Message

def generate_response_series_from_faq_test_set(test_df: pd.DataFrame) -> tuple[pd.Series, pd.Series]:
    responses = []
    retrieved_contexts = []
    for index, row in test_df.iterrows():
        question = row["user_input"]
        answer = gen_answer(
            UUID("f6f200d4-e309-47fc-8cc2-ffac77cdb8ad"),
            uuid4(),
            [Message(content=question, author="user")],
        )

        tool_call_correct = False
        for key, value in answer.metadata.items():
            if key == "search_faq_database_tool":
                retrieved_contexts.append([value])
                tool_call_correct = True
        if not tool_call_correct:
            retrieved_contexts.append([""])
        responses.append(answer.content)
    return (
        pd.Series(responses, name="response"),
        pd.Series(retrieved_contexts, name="retrieved_contexts"),
    )

response_series, retrieved_context_series = generate_response_series_from_faq_test_set(
    faq_test_df
)

faq_test_df["response"] = response_series
faq_test_df["retrieved_contexts"] = retrieved_context_series

2024-12-07 15:06:50 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
counter: 1


==== tool_calls ====


search_faq_database_tool
2024-12-07 15:06:50,633 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-07 15:06:50 - BEGIN (implicit)
2024-12-07 15:06:50,638 INFO sqlalchemy.engine.Engine SELECT user_memory.id, user_memory.user_id, user_memory.thread_id, user_memory.user_demand, user_memory.product_name, user_memory.brand, user_memory.min_price, user_memory.max_price, user_memory.phone_number, user_memory.email, user_memory.created_at, user_memory.updated_at 
FROM user_memory 
WHERE user_memory.thread_id = %(thread_id_1)s::UUID
2024-12-07 15:06:50 - SELECT user_memory.id, user_memory.user_id, user_memory.thread_id, user_memory.user_demand, user_memory.product_name, user_memory.brand, user_memory.min_price, user_memory.max_price, user_memory.phone_number, user_memory.email, user_memory.created_at, user_memory.updated_at 
FROM user_memory 
WHERE user_

In [7]:
faq_test_df.head()

,user_input,response,retrieved_contexts,reference
0,FPT Shop thu thập thông tin cá nhan cua khach ...,FPT Shop thu thập thông tin cá nhân của khách ...,"[""- Title: Mục đích và phạm vi thu thập\n - C...",FPTshop.com.vn thu thập thông tin cá nhân của ...
1,How does FPTshop.com.vn use customer personal ...,FPTshop.com.vn thu thập thông tin cá nhân của ...,"[""- Title: Mục đích và phạm vi thu thập\n - C...",FPTshop.com.vn chỉ sử dụng thông tin cá nhân c...
2,Thông tin cá nhân của khách hàng được lưu trữ ...,Thông tin cá nhân của khách hàng sẽ được lưu t...,"[""- Title: Thời gian lưu trữ thông tin\n - Câ...",Dữ liệu cá nhân sẽ được lưu trữ đến khi có yêu...
3,FPT Shop cam kết bảo vệ thông tin của khách hà...,FPT Shop cam kết bảo mật thông tin cá nhân của...,"[""- Title: Cam kết bảo mật thông tin\n - Câu ...",FPTshop.com.vn cam kết bảo mật thông tin cá nh...
4,"As a Data Privacy Officer, I am particularly i...",FPTShop đảm bảo rằng thông tin thẻ thanh toán ...,"[""- Title: Quy định bảo mật khi thanh toán\n ...",FPTShop ensures that customer payment card inf...


In [33]:
from ragas import EvaluationDataset

faq_eval_dataset = EvaluationDataset.from_pandas(faq_test_df)

In [50]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from service.embedding import _model as model

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", max_retries=15, timeout=30))
evaluator_embeddings = LangchainEmbeddingsWrapper(
    OpenAIEmbeddings(model=model, max_retries=5, timeout=30)
)

In [36]:
from ragas.metrics import AnswerRelevancy, Faithfulness, ContextRecall, ContextPrecision, AnswerSimilarity, AnswerCorrectness
from ragas import evaluate

metrics = [
    AnswerRelevancy(embeddings=evaluator_embeddings),
    Faithfulness(llm=evaluator_llm),
    ContextRecall(llm=evaluator_llm),
    ContextPrecision(llm=evaluator_llm),
    AnswerSimilarity(embeddings=evaluator_embeddings),
    AnswerCorrectness(embeddings=evaluator_embeddings),
]
results = evaluate(dataset=faq_eval_dataset, metrics=metrics)

Evaluating:   0%|          | 0/162 [00:00<?, ?it/s]

2024-12-07 15:09:36 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-12-07 15:09:36 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-12-07 15:09:37 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Evaluating:   1%|          | 1/162 [00:00<02:31,  1.06it/s]

2024-12-07 15:09:37 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-07 15:09:37 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-07 15:09:37 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-07 15:09:37 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Evaluating:   1%|          | 2/162 [00:01<01:14,  2.16it/s]

...

Evaluating:  99%|█████████▉| 160/162 [01:01<00:01,  1.67it/s]

2024-12-07 15:10:38 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-07 15:10:38 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-12-07 15:10:38 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-12-07 15:10:38 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Evaluating:  99%|█████████▉| 161/162 [01:02<00:00,  1.54it/s]

2024-12-07 15:10:41 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Evaluating: 100%|██████████| 162/162 [01:05<00:00,  2.48it/s]


In [37]:
results_df = results.to_pandas()
results_df.to_csv("faq_evaluation_results.csv", index=False)

In [38]:
results_df.head()

,user_input,retrieved_contexts,response,reference,answer_relevancy,faithfulness,context_recall,context_precision,semantic_similarity,answer_correctness
0,FPT Shop thu thập thông tin cá nhan cua khach ...,[- Title: Mục đích và phạm vi thu thập\n - Câ...,FPT Shop thu thập thông tin cá nhân của khách ...,FPTshop.com.vn thu thập thông tin cá nhân của ...,0.547345,0.850000,1.0,1.0,0.877220,0.969304
1,How does FPTshop.com.vn use customer personal ...,[- Title: Mục đích và phạm vi thu thập\n - Câ...,FPTshop.com.vn thu thập thông tin cá nhân của ...,FPTshop.com.vn chỉ sử dụng thông tin cá nhân c...,0.929337,0.888889,1.0,1.0,0.810529,0.202632
2,Thông tin cá nhân của khách hàng được lưu trữ ...,[- Title: Thời gian lưu trữ thông tin\n - Câu...,Thông tin cá nhân của khách hàng sẽ được lưu t...,Dữ liệu cá nhân sẽ được lưu trữ đến khi có yêu...,0.583729,0.600000,1.0,1.0,0.863675,0.965914
3,FPT Shop cam kết bảo vệ thông tin của khách hà...,[- Title: Cam kết bảo mật thông tin\n - Câu h...,FPT Shop cam kết bảo mật thông tin cá nhân của...,FPTshop.com.vn cam kết bảo mật thông tin cá nh...,0.711569,0.600000,1.0,1.0,0.860205,0.715053
4,"As a Data Privacy Officer, I am particularly i...",[- Title: Quy định bảo mật khi thanh toán\n -...,FPTShop đảm bảo rằng thông tin thẻ thanh toán ...,FPTShop ensures that customer payment card inf...,0.723454,0.833333,1.0,1.0,0.707756,0.476939


In [39]:
faq_scores_df = results_df[["answer_relevancy","faithfulness","context_recall","context_precision","semantic_similarity","answer_correctness"]]
faq_scores_df.mean()

answer_relevancy       0.629268
faithfulness           0.714415
context_recall         0.962963
context_precision      1.000000
semantic_similarity    0.765779
answer_correctness     0.714481
dtype: float64

In [40]:
import pandas as pd

phone_test_df = pd.read_csv("phone_testset.csv")

In [41]:
phone_test_df

,user_input,reference
0,Tecno camon 30 8GB là điện thoại gì ?,Tecno Camon 30 8GB là một chiếc điện thoại với...
1,Mô tả về điện thoại Mobell F209,Điện thoại Mobell F209 4G là một lựa chọn phù ...
2,Giá hiện tại của Xiaomi 14T Pro là bao nhiêu?,"Xin lỗi, mình không tìm thấy thông tin về giá ..."
3,Các phiên bản màu của iPhone 14 hiện có tại cử...,iPhone 14 hiện có các phiên bản màu sắc sau tạ...
4,Có mẫu điện thoại Samsung có giá dưới 10 triệu...,"Tại cửa hàng FPTShop, có một số mẫu điện thoại..."
...,...,...
109,Ưu đãi khi mua Masstel Izi T2 là gì?,"Khi mua điện thoại Masstel Izi T2, bạn có thể ..."
110,Khuyến mãi khi mua samsung a23,"Xin lỗi, mình không có thông tin về khuyến mãi..."
111,Cung cấp thông tin chi tiết về Mobell M331,Điện thoại Mobell M331 4G có giá 550.000 VND. ...
112,Honor 90 Lite hiện có khuyến mãi nào khi mua k...,"Hiện tại, khi mua Honor 90 Lite, bạn có thể nh..."


In [44]:
from service.store_chatbot import gen_answer
from uuid import uuid4, UUID
from models.message import Message

def generate_response_series_from_phone_test_set(
    test_df: pd.DataFrame,
) -> tuple[pd.Series, pd.Series]:
    responses = []
    retrieved_contexts = []
    for index, row in test_df.iterrows():
        question = row["user_input"]
        answer = gen_answer(
            UUID("f6f200d4-e309-47fc-8cc2-ffac77cdb8ad"),
            uuid4(),
            [Message(content=question, author="user")],
        )

        tool_call_correct = False
        for key, value in answer.metadata.items():
            if key == "search_phone_database_tool":
                tool_call_correct = True
                retrieved_contexts.append([value])
        
        if not tool_call_correct:
            retrieved_contexts.append([""])
        
        responses.append(answer.content)
    return (
        pd.Series(responses, name="response"),
        pd.Series(retrieved_contexts, name="retrieved_contexts"),
    )


response_series, retrieved_context_series = (
    generate_response_series_from_phone_test_set(phone_test_df)
)

phone_test_df["response"] = response_series
phone_test_df["retrieved_contexts"] = retrieved_context_series

2024-12-07 15:15:44 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
counter: 1


==== tool_calls ====


collect_and_update_user_requirements_tool
2024-12-07 15:15:44,965 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-07 15:15:44 - BEGIN (implicit)
2024-12-07 15:15:44,969 INFO sqlalchemy.engine.Engine SELECT user_memory.id, user_memory.user_id, user_memory.thread_id, user_memory.user_demand, user_memory.product_name, user_memory.brand, user_memory.min_price, user_memory.max_price, user_memory.phone_number, user_memory.email, user_memory.created_at, user_memory.updated_at 
FROM user_memory 
WHERE user_memory.thread_id = %(thread_id_1)s::UUID
2024-12-07 15:15:44 - SELECT user_memory.id, user_memory.user_id, user_memory.thread_id, user_memory.user_demand, user_memory.product_name, user_memory.brand, user_memory.min_price, user_memory.max_price, user_memory.phone_number, user_memory.email, user_memory.created_at, user_memory.updated_at 
FROM user_me

In [45]:
phone_test_df.head()

,user_input,reference,response,retrieved_contexts
0,Tecno camon 30 8GB là điện thoại gì ?,Tecno Camon 30 8GB là một chiếc điện thoại với...,**Tecno Camon 30 8GB** là một chiếc điện thoại...,[## Below is the information of the phone prod...
1,Mô tả về điện thoại Mobell F209,Điện thoại Mobell F209 4G là một lựa chọn phù ...,Điện thoại Mobell F209 4G là một lựa chọn lý t...,[## Below is the information of the phone prod...
2,Giá hiện tại của Xiaomi 14T Pro là bao nhiêu?,"Xin lỗi, mình không tìm thấy thông tin về giá ...","Xin lỗi, mình không tìm thấy thông tin về điện...","[You must say: 'I'm sorry, I couldn't find the..."
3,Các phiên bản màu của iPhone 14 hiện có tại cử...,iPhone 14 hiện có các phiên bản màu sắc sau tạ...,iPhone 14 hiện có tại cửa hàng với các phiên b...,[## Below is the information of the phone prod...
4,Có mẫu điện thoại Samsung có giá dưới 10 triệu...,"Tại cửa hàng FPTShop, có một số mẫu điện thoại...",Dưới đây là một số mẫu điện thoại Samsung có g...,[## Below is the information of the phone prod...


In [51]:
from ragas.metrics import (
    AnswerRelevancy,
    Faithfulness,
    ContextRecall,
    ContextPrecision,
    AnswerSimilarity,
    AnswerCorrectness,
)
from ragas import evaluate

metrics = [
    AnswerRelevancy(embeddings=evaluator_embeddings),
    Faithfulness(llm=evaluator_llm),
    ContextRecall(llm=evaluator_llm),
    ContextPrecision(llm=evaluator_llm),
    AnswerSimilarity(embeddings=evaluator_embeddings),
    AnswerCorrectness(embeddings=evaluator_embeddings),
]

phone_eval_dataset = EvaluationDataset.from_pandas(phone_test_df)
results = evaluate(dataset=phone_eval_dataset, metrics=metrics)

Evaluating:   0%|          | 0/684 [00:00<?, ?it/s]

2024-12-07 15:39:11 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-12-07 15:39:11 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-07 15:39:11 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-07 15:39:11 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-07 15:39:11 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-07 15:39:11 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Evaluating:   0%|          | 1/684 [00:01<18:33,  1.63s/it]

2024-12-07 15:39:12 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Evaluating:   0%|          | 3/684 [00:01<05:25,  2.09it/s]

...

Evaluating: 100%|█████████▉| 682/684 [07:53<00:03,  1.82s/it]

2024-12-07 15:47:04 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-12-07 15:47:05 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Evaluating: 100%|█████████▉| 683/684 [07:54<00:01,  1.61s/it]

2024-12-07 15:47:05 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Evaluating: 100%|██████████| 684/684 [07:54<00:00,  1.44it/s]


In [55]:
results_df = results.to_pandas()
results_df.to_csv("phone_evaluation_results.csv", index=False)

In [56]:
results_df

,user_input,retrieved_contexts,response,reference,answer_relevancy,faithfulness,context_recall,context_precision,semantic_similarity,answer_correctness
0,Tecno camon 30 8GB là điện thoại gì ?,[## Below is the information of the phone prod...,**Tecno Camon 30 8GB** là một chiếc điện thoại...,Tecno Camon 30 8GB là một chiếc điện thoại với...,0.622160,1.0,1.0,1.0,0.936172,0.702794
1,Mô tả về điện thoại Mobell F209,[## Below is the information of the phone prod...,Điện thoại Mobell F209 4G là một lựa chọn lý t...,Điện thoại Mobell F209 4G là một lựa chọn phù ...,0.662360,1.0,1.0,1.0,0.928857,0.665556
2,Giá hiện tại của Xiaomi 14T Pro là bao nhiêu?,"[You must say: 'I'm sorry, I couldn't find the...","Xin lỗi, mình không tìm thấy thông tin về điện...","Xin lỗi, mình không tìm thấy thông tin về giá ...",0.000000,0.5,1.0,0.0,0.897649,0.474407
3,Các phiên bản màu của iPhone 14 hiện có tại cử...,[## Below is the information of the phone prod...,iPhone 14 hiện có tại cửa hàng với các phiên b...,iPhone 14 hiện có các phiên bản màu sắc sau tạ...,0.785356,1.0,1.0,1.0,0.862206,0.590612
4,Có mẫu điện thoại Samsung có giá dưới 10 triệu...,[## Below is the information of the phone prod...,Dưới đây là một số mẫu điện thoại Samsung có g...,"Tại cửa hàng FPTShop, có một số mẫu điện thoại...",0.754269,1.0,1.0,1.0,0.930668,0.832671
...,...,...,...,...,...,...,...,...,...,...
109,Ưu đãi khi mua Masstel Izi T2 là gì?,[## Below is the information of the phone prod...,"Hiện tại, khi mua điện thoại Masstel Izi T2, b...","Khi mua điện thoại Masstel Izi T2, bạn có thể ...",0.687609,1.0,1.0,1.0,0.949092,0.987273
110,Khuyến mãi khi mua samsung a23,[## Below is the information of the phone prod...,"Hiện tại, Samsung Galaxy A23 5G đang có giá kh...","Xin lỗi, mình không có thông tin về khuyến mãi...",0.601539,1.0,0.0,1.0,0.770186,0.192510
111,Cung cấp thông tin chi tiết về Mobell M331,[## Below is the information of the phone prod...,Thông tin chi tiết về điện thoại Mobell M331 4...,Điện thoại Mobell M331 4G có giá 550.000 VND. ...,0.691893,1.0,0.8,1.0,0.885114,0.506993
112,Honor 90 Lite hiện có khuyến mãi nào khi mua k...,[## Below is the information of the phone prod...,"Hiện tại, Honor 90 Lite đang có một số khuyến ...","Hiện tại, khi mua Honor 90 Lite, bạn có thể nh...",0.729052,1.0,1.0,1.0,0.953284,0.944203


In [57]:
phone_scores_df = results_df[
    [
        "answer_relevancy",
        "faithfulness",
        "context_recall",
        "context_precision",
        "semantic_similarity",
        "answer_correctness",
    ]
]
phone_scores_df.mean()

answer_relevancy       0.501901
faithfulness           0.884721
context_recall         0.911705
context_precision      0.848214
semantic_similarity    0.892521
answer_correctness     0.632421
dtype: float64